In [14]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [15]:
import pickle
import json
from rich import print

In [16]:
# Condif With Feature names
with open('src/config.json', 'r') as f:
    config_dict = json.load(f)

y_column = config_dict['y_column']
numeric_features = config_dict['numeric_features']
categorical_features = config_dict['categorical_features']
all_features = numeric_features + categorical_features

# Processed Data
with open('data/processed/ml_data.pkl', 'rb') as f:
    data_dict = pickle.load(f)

X_train = data_dict['X_train']
X_test = data_dict['X_test']
y_train = data_dict['y_train']
y_test = data_dict['y_test']
feature_names = data_dict['feature_names']
X_preprocessor = data_dict['X_preprocessor']
y_preprocessor = data_dict['y_preprocessor']

In [17]:
X_train

array([[ 0.13907964,  0.13907964, -0.00963163, ...,  0.        ,
         0.        ,  0.        ],
       [-0.50730151, -0.50730151, -0.01059335, ...,  0.        ,
         0.        ,  0.        ],
       [-0.64581176, -0.64581176, -0.01022832, ...,  0.        ,
         0.        ,  0.        ],
       ...,
       [-0.73815192, -0.73815192, -0.01059335, ...,  0.        ,
         0.        ,  0.        ],
       [ 1.459544  ,  1.459544  , -0.00862779, ...,  0.        ,
         0.        ,  0.        ],
       [-0.57193963, -0.57193963, -0.01017216, ...,  0.        ,
         0.        ,  0.        ]], shape=(20406, 35))

In [18]:
X_test

array([[ 0.86856695,  0.86856695, -0.01008792, ...,  0.        ,
         0.        ,  0.        ],
       [-0.2764511 , -0.2764511 , -0.01059335, ...,  0.        ,
         0.        ,  0.        ],
       [ 1.15482146,  1.15482146, -0.01059335, ...,  0.        ,
         0.        ,  0.        ],
       ...,
       [-0.56270561, -0.56270561, -0.01020726, ...,  0.        ,
         0.        ,  0.        ],
       [-0.02713265, -0.02713265, -0.00975799, ...,  0.        ,
         0.        ,  0.        ],
       [ 2.2444354 ,  2.2444354 , -0.00803111, ...,  0.        ,
         0.        ,  0.        ]], shape=(5102, 35))

### Entrenamiento

In [20]:
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from sklearn.linear_model import ElasticNet

from src.evaluate import train_eval_model
from src.training import (
    optimize_hyperparameters,
    objective_random_forest,
    objective_lightgbm,
    objective_xgboost,
    objective_elastic_net
) 

models = []
scores = []
print('Tunning Random Forest')
best_params, score = optimize_hyperparameters(objective_random_forest, X_train, y_train.ravel())

models.append(RandomForestRegressor(**best_params))
scores.append(score)

print()
print('Tunning XGBoost')
best_params, score = optimize_hyperparameters(objective_xgboost, X_train, y_train.ravel())
models.append(XGBRegressor(**best_params))
scores.append(score)

print()
print('Tunning LGBoost')
best_params, score = optimize_hyperparameters(objective_lightgbm, X_train, y_train.ravel())
models.append(LGBMRegressor(**best_params))
scores.append(score)

print()
print('Tunning Elastic Net')
best_params, score = optimize_hyperparameters(objective_elastic_net, X_train, y_train.ravel())
models.append(ElasticNet(**best_params))
scores.append(score)

train_eval_model(models, scores, X_train, y_train, X_test, y_test, y_preprocessor = y_preprocessor)


Tunning Random Forest

Best trial: 2. Best value: -0.0697623: 100%|██████████| 4/4 [01:22<00:00, 20.74s/it]

 Best score : -0.06976233472359877


Tunning XGBoost

Best trial: 1. Best value: -0.0956472: 100%|██████████| 4/4 [00:06<00:00,  1.65s/it]

...

Tunning Elastic Net

Best trial: 3. Best value: -0.223064: 100%|██████████| 4/4 [00:01<00:00,  2.58it/s]
/home/golo/Documents/Proyectos/InmueblesApp/.venv/lib64/python3.13/site-packages/sklearn/utils/validation.py:1406: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


 Best score : -0.22306388430674423
Pipeline(steps=[('log_transform',
                 FunctionTransformer(func=<ufunc 'log'>,
                                     inverse_func=<ufunc 'exp'>)),
                ('scaler', StandardScaler())])
[LightGBM] [Warning] feature_fraction is set=0.9878408763353723, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9878408763353723
[LightGBM] [Warning] bagging_fraction is set=0.9978162800060256, subsample=1.0 will be ignored. Current value: bagging_fraction=0.9978162800060256
[LightGBM] [Warning] bagging_freq is set=2, subsample_freq=0 will be ignored. Current value: bagging_freq=2
[LightGBM] [Warning] feature_fraction is set=0.9878408763353723, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9878408763353723
[LightGBM] [Warning] bagging_fraction is set=0.9978162800060256, subsample=1.0 will be ignored. Current value: bagging_fraction=0.9978162800060256
[LightGBM] [Warning] bagging_freq is set=2, subsam

/home/golo/Documents/Proyectos/InmueblesApp/.venv/lib64/python3.13/site-packages/sklearn/utils/validation.py:2749: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(
/home/golo/Documents/Proyectos/InmueblesApp/.venv/lib64/python3.13/site-packages/sklearn/preprocessing/_function_transformer.py:387: RuntimeWarning: overflow encountered in exp
  return func(X, **(kw_args if kw_args else {}))


ValueError: Input contains infinity or a value too large for dtype('float64').

In [ ]:
models

[RandomForestRegressor(max_depth=13, min_samples_leaf=5, min_samples_split=8,
                       n_estimators=938),
 XGBRegressor(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.7833698902429476, device=None,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric=None, feature_types=None, feature_weights=None,
              gamma=1.7909957292454788, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.05025623564664764,
              max_bin=None, max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=11, max_leaves=None,
              min_child_weight=6, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=393, n_jobs=None,
              num_parallel_tree=None, ...),
 LGBMRegressor(bagging_fraction=0.8244526567886715, bagging_freq=2,

In [ ]:
scores

[-0.07098610116267982,
 -0.09724216356656995,
 -0.06897068383900888,
 -0.2207679582455687]

In [ ]:
X_train.shape

(20406, 35)

In [ ]:
X_test

array([[ 0.86856695,  0.86856695, -0.01008792, ...,  0.        ,
         0.        ,  0.        ],
       [-0.2764511 , -0.2764511 , -0.01059335, ...,  0.        ,
         0.        ,  0.        ],
       [ 1.15482146,  1.15482146, -0.01059335, ...,  0.        ,
         0.        ,  0.        ],
       ...,
       [-0.56270561, -0.56270561, -0.01020726, ...,  0.        ,
         0.        ,  0.        ],
       [-0.02713265, -0.02713265, -0.00975799, ...,  0.        ,
         0.        ,  0.        ],
       [ 2.2444354 ,  2.2444354 , -0.00803111, ...,  0.        ,
         0.        ,  0.        ]], shape=(5102, 35))

In [ ]:
y_test

,PRICE
15389,445000000
7885,590000000
9853,1750000000
4036,210000000
17536,4200000000
...,...
5516,2000000000
13141,263000000
18737,320000000
618,650000000


In [ ]:
y_preprocessor

,steps,"[('log_transform', ...), ('scaler', ...)]"
,transform_input,None
,memory,None
,verbose,False
,func,<ufunc 'log'>
,inverse_func,<ufunc 'exp'>
,validate,False
,accept_sparse,False
,check_inverse,True
,feature_names_out,None
,kw_args,None


In [ ]:
train_eval_model(models, scores, X_train, y_train, X_test, y_test, y_preprocessor = y_preprocessor)

Pipeline(steps=[('log_transform',
                 FunctionTransformer(func=<ufunc 'log'>,
                                     inverse_func=<ufunc 'exp'>)),
                ('scaler', StandardScaler())])
[LightGBM] [Warning] feature_fraction is set=0.7073626430344391, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.7073626430344391
[LightGBM] [Warning] bagging_fraction is set=0.8244526567886715, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8244526567886715
[LightGBM] [Warning] bagging_freq is set=2, subsample_freq=0 will be ignored. Current value: bagging_freq=2
[LightGBM] [Warning] feature_fraction is set=0.7073626430344391, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.7073626430344391
[LightGBM] [Warning] bagging_fraction is set=0.8244526567886715, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8244526567886715
[LightGBM] [Warning] bagging_freq is set=2, subsample_freq=0 will be ignored. Current

/home/golo/Documents/Proyectos/InmueblesApp/.venv/lib64/python3.13/site-packages/sklearn/utils/validation.py:1406: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

/home/golo/Documents/Proyectos/InmueblesApp/.venv/lib64/python3.13/site-packages/sklearn/utils/validation.py:2749: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(
/home/golo/Documents/Proyectos/InmueblesApp/.venv/lib64/python3.13/site-packages/sklearn/preprocessing/_function_transformer.py:387: RuntimeWarning: overflow encountered in exp
  return func(X, **(kw_args if kw_args else {}))


ValueError: Input contains infinity or a value too large for dtype('float64').

In [ ]:
type(y_preprocessor)

sklearn.pipeline.Pipeline